# DS-SF-23 | Codealong 02 | Tidying up (more) the SF housing dataset

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 6)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

In [2]:
df = pd.read_csv(os.path.join('..', 'datasets', 'zillow-02-start.csv'), index_col = 'ID')

> ## Remove columns

In [3]:
df.drop(['Latitude', 'Longitude'], axis = 1, inplace = True)

df

,Address,DateOfSale,SalePrice,SalePriceUnit,IsAStudio,...,Size,SizeUnit,LotSize,LotSizeUnit,BuiltInYear
ID,,,,,,,,,,,
15063471,"55 Vandewater St APT 9, San Francisco, CA",12/4/15,710000.00,$,False,...,550.0,sqft,NaN,NaN,1980.0
15063505,"740 Francisco St, San Francisco, CA",11/30/15,2.15,$M,False,...,1430.0,sqft,2435.0,sqft,1948.0
15063609,"819 Francisco St, San Francisco, CA",11/12/15,5.60,$M,False,...,2040.0,sqft,3920.0,sqft,1976.0
...,...,...,...,...,...,...,...,...,...,...,...
2128308939,"33 Santa Cruz Ave, San Francisco, CA",12/10/15,830000.00,$,False,...,1738.0,sqft,2299.0,sqft,1976.0
2131957929,"1821 Grant Ave, San Francisco, CA",12/15/15,835000.00,$,False,...,1048.0,sqft,NaN,NaN,1975.0
2136213970,"1200 Gough St, San Francisco, CA",1/10/16,825000.00,$,False,...,900.0,sqft,NaN,NaN,1966.0


Converting All Prices to Dollars

In [4]:
df.SalePriceUnit.unique()

array(['$', '$M'], dtype=object)

In [4]:
df_1 = df[df.SalePriceUnit == '$']

df_1 = df_1.drop('SalePriceUnit', axis = 1)
# Really a workaround as DataFrame.drop() with inplace = True issue warnings...

df_6 = df[df.SalePriceUnit == '$M']

df_6 = df_6.drop('SalePriceUnit', axis = 1)

# Replacing the content of a column
df_6.SalePrice = df_6.SalePrice * (10 ** 6)

# Adding rows to a DataFrame
# Concatenation of two DataFrame objects
df = pd.concat([df_1, df_6])

In [6]:
df.sort_index()

,Address,DateOfSale,SalePrice,IsAStudio,BedCount,...,Size,SizeUnit,LotSize,LotSizeUnit,BuiltInYear
ID,,,,,,,,,,,
15063471,"55 Vandewater St APT 9, San Francisco, CA",12/4/15,710000.0,False,1.0,...,550.0,sqft,NaN,NaN,1980.0
15063505,"740 Francisco St, San Francisco, CA",11/30/15,2150000.0,False,NaN,...,1430.0,sqft,2435.0,sqft,1948.0
15063609,"819 Francisco St, San Francisco, CA",11/12/15,5600000.0,False,2.0,...,2040.0,sqft,3920.0,sqft,1976.0
...,...,...,...,...,...,...,...,...,...,...,...
2128308939,"33 Santa Cruz Ave, San Francisco, CA",12/10/15,830000.0,False,3.0,...,1738.0,sqft,2299.0,sqft,1976.0
2131957929,"1821 Grant Ave, San Francisco, CA",12/15/15,835000.0,False,2.0,...,1048.0,sqft,NaN,NaN,1975.0
2136213970,"1200 Gough St, San Francisco, CA",1/10/16,825000.0,False,1.0,...,900.0,sqft,NaN,NaN,1966.0


> ## Converting studio from a boolean to a binary variable (0 or 1)

In [15]:
df['IsAStudio'] = 1 * df['IsAStudio']

In [16]:
type(df['IsAStudio'])

pandas.core.series.Series

> ## Size

In [17]:
df.SizeUnit.unique()

AttributeError: 'DataFrame' object has no attribute 'SizeUnit'

In [18]:
df.SizeUnit.value_counts()

AttributeError: 'DataFrame' object has no attribute 'SizeUnit'

Size is either in square feet or missing.  Almost no work needed except to remove size unit.

In [12]:
df.drop('SizeUnit', axis = 1, inplace = True)

> ## Lot Size

In [19]:
df.LotSizeUnit.unique()

array([nan, 'sqft', 'ac'], dtype=object)

Lot sizes are either in square feet or in acres.  Let's convert them all to square feet.

In [20]:
df_na = df[df.LotSizeUnit.isnull()]
df_na = df_na.drop('LotSizeUnit', axis = 1)
len(df_na)

444

In [32]:
new_df = pd.DataFrame({'column': [np.nan, 1]})
new_df
new_df.column.isnull()

0     True
1    False
Name: column, dtype: bool

False

In [22]:
df_sqft = df[df.LotSizeUnit == 'sqft']
df_sqft = df_sqft.drop('LotSizeUnit', axis = 1)
len(df_sqft)

552

In [21]:
df_ac = df[df.LotSizeUnit == 'ac']
df_ac = df_ac.drop('LotSizeUnit', axis = 1)
len(df_ac)

4

In [15]:
# 1 acre = 43560 sqft

df_ac.LotSize = df_ac.LotSize * (43560)

In [16]:
df = pd.concat([df_na, df_sqft, df_ac]).sort_index()

In [17]:
df

,Address,DateOfSale,SalePrice,IsAStudio,BedCount,BathCount,Size,LotSize,BuiltInYear
ID,,,,,,,,,
15063471,"55 Vandewater St APT 9, San Francisco, CA",12/4/15,710000.0,0,1.0,NaN,550.0,NaN,1980.0
15063505,"740 Francisco St, San Francisco, CA",11/30/15,2150000.0,0,NaN,2.0,1430.0,2435.0,1948.0
15063609,"819 Francisco St, San Francisco, CA",11/12/15,5600000.0,0,2.0,3.5,2040.0,3920.0,1976.0
...,...,...,...,...,...,...,...,...,...
2128308939,"33 Santa Cruz Ave, San Francisco, CA",12/10/15,830000.0,0,3.0,3.0,1738.0,2299.0,1976.0
2131957929,"1821 Grant Ave, San Francisco, CA",12/15/15,835000.0,0,2.0,2.0,1048.0,NaN,1975.0
2136213970,"1200 Gough St, San Francisco, CA",1/10/16,825000.0,0,1.0,1.0,900.0,NaN,1966.0


> ## Save the `DataFrame` to a `.csv` file

In [33]:
df.to_csv(os.path.join('..', 'datasets', 'zillow-02-finish.csv'), index_label = 'ID')